In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_absolute_error
from random import randint
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, rand

train = pd.read_csv('./data/allstate/train.csv')
test = pd.read_csv('./data/allstate/test.csv')

test['loss'] = np.nan
joined = pd.concat([train, test])

def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    con = 2
    x =preds-labels
    grad =con*x / (np.abs(x)+con)
    hess =con**2 / (np.abs(x)+con)**2
    return grad, hess 

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))


cat_feature = [n for n in joined.columns if n.startswith('cat')]    
cont_feature = [n for n in joined.columns if n.startswith('cont')] 

for column in cat_feature:
        joined[column] = pd.factorize(joined[column].values, sort=True)[0]

    
train = joined[joined['loss'].notnull()]
test = joined[joined['loss'].isnull()]
shift = 202
y = np.log(train['loss'] + shift)
ids = test['id']
X = train.drop(['loss', 'id'], 1)
X_test = test.drop(['loss', 'id'], 1)
xgfull = xgb.DMatrix(X, label=y)

def score(params):           
    print("Training with params : ")
    print(params)

    final_fold_prediction= []
    final_fold_real = []

    prediction = np.zeros(X.shape[0])
    
    n_folds  = params["n_folds"]
    del(params["n_folds"])
    
    kf = KFold(X.shape[0], n_folds=n_folds)
    
    for i, (train_index, test_index) in enumerate(kf):        
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]    
          
        xgtrain = xgb.DMatrix(X_train, label=y_train)
        xgtrain_2 = xgb.DMatrix(X_val, label=y_val)
        
        watchlist = [(xgtrain, 'train'), (xgtrain_2, 'eval')]                    

        model = xgb.train(params, xgtrain, 100000, watchlist, obj=logregobj, feval=evalerror, 
        early_stopping_rounds=300, verbose_eval=200)        

        X_val = xgb.DMatrix(X_val) 
        temp_serises = pd.Series(np.exp(model.predict(X_val))-shift)
        final_fold_prediction.append( temp_serises )
        temp_serises = np.exp(y_val) -shift
        final_fold_real.append(temp_serises )
        
        temp_cv_score = mean_absolute_error(np.exp(model.predict(X_val))-shift, np.exp(y_val) -shift)
        print("Fold {} score: {}".format(i + 1, temp_cv_score))
        
        prediction += np.exp(model.predict(xgfull)) - shift

    prediction = prediction/n_folds
    score =  mean_absolute_error(y, prediction)
        
    print("\tMAE {0}\n\n".format(score))    
    return {'loss': score, 'status': STATUS_OK}

def optimize(trials):
    RANDOM_STATE = 2016
    space = {
             'min_child_weight' : hp.quniform('min_child_weight', 0.01, 1.0, 0.01),
             'eta' : hp.quniform('eta', 0.001, 1.0, 0.01),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.01, 1.0, 0.01),
             'max_depth' : hp.choice('batch_size', np.arange(1, 20, dtype=int)),   
             'subsample' : hp.quniform('subsample', 0.01, 1.0, 0.01),
             'alpha' : hp.quniform('alpha', 0.01, 1.0, 0.01),
             'gamma' : hp.quniform('gamma', 0.01, 1.0, 0.01),                        
             'seed' : RANDOM_STATE,
             'n_folds' : hp.choice('n_folds', np.arange(2, 10, dtype=int)), 
             }

    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=250)

    print("Best params predicted by hyperopt are:")    
    print(best)

#Trials object where the history of search will be stored
trials = Trials()

optimize(trials)

/home/arvc/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Training with params : 
{'seed': 2016, 'max_depth': 8, 'eta': 0.08, 'subsample': 0.59, 'gamma': 0.67, 'min_child_weight': 0.1, 'alpha': 0.36, 'n_folds': 9, 'colsample_bytree': 0.64}
[0]	train-mae:3212.42	eval-mae:3228.95
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 300 rounds.
[200]	train-mae:1051.42	eval-mae:1164.25
[400]	train-mae:997.237	eval-mae:1161.89
[600]	train-mae:958.671	eval-mae:1163.49
Stopping. Best iteration:
[368]	train-mae:1003.76	eval-mae:1160.9

Fold 1 score: 1165.2361656730695
[0]	train-mae:3216.95	eval-mae:3192.67
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 300 rounds.
[200]	train-mae:1052.28	eval-mae:1142.67
[400]	train-mae:998.418	eval-mae:1139.47
[600]	train-mae:961.788	eval-mae:1141.11
Stopping. Best iteration:
[471]	train-mae:984.085	eval-mae:1139.07

Fold 2 score: 1143.6871211923396
[0]	train

/home/arvc/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:26: RuntimeWarning: overflow encountered in exp


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').